In [18]:
import csv
import pandas as pd
import numpy as np
import os
import subprocess
from pathlib import Path

In [19]:
def main():
    exp_dirs = (Path('.')/'results'/'exp1').glob('baseline14')
    for exp_dir in exp_dirs:
        print(exp_dir)
        for results_dir in exp_dir.glob('*'):
            print(results_dir.name)
            # results_dir_split = results_dir.name.split('_')
            resultsDirName = results_dir.name
            for csv_file_path in results_dir.glob('*'):
                print(csv_file_path.stem.split('_')[-1])
                poseNum = csv_file_path.stem.split('_')[-1]
                df = pd.read_csv(csv_file_path)
                df['vmaf'].mean()
                # print(df.head(1))
            

In [20]:
if __name__ == '__main__':
    main()

results/exp1/baseline14
idF5_16users_baseline14_slvrbaseline_ds5_L12_k3_ch32_r1.0_thr0.9
pose15
pose14
pose13
pose12
pose10
pose11
pose9
pose8
pose6
pose7
pose5
pose4
pose0
pose1
pose3
pose2
idF5_16users_baseline14_slvrbaseline_ds5_L6_k3_ch32_r1.0_thr0.9
pose6
pose7
pose5
pose4
pose0
pose1
pose3
pose2
pose15
pose14
pose10
pose9
pose8
pose11
pose13
pose12
idF5_16users_baseline14_slvrbaseline_ds5_L12_k3_ch32_r1.0_thr0.7
pose4
pose5
pose7
pose6
pose2
pose3
pose1
pose0
pose8
pose9
pose11
pose10
pose12
pose13
pose14
pose15
idF5_16users_baseline14_slvrbaseline_ds5_L6_k3_ch32_r1.0_thr0.7
pose12
pose13
pose11
pose10
pose14
pose15
pose8
pose9
pose4
pose5
pose7
pose6
pose2
pose3
pose1
pose0
idF5_16users_baseline14_slvrbaseline_ds5_L8_k3_ch32_r1.0_thr0.9
pose7
pose6
pose4
pose15
pose14
pose5
pose1
pose10
pose11
pose0
pose2
pose13
pose12
pose3
pose8
pose9
idF5_16users_baseline14_slvrbaseline_ds5_L8_k3_ch32_r1.0_thr0.7
pose9
pose8
pose5
pose12
pose13
pose4
pose6
pose11
pose10
pose7
pose3
pose14
pos